<a href="https://colab.research.google.com/github/Kidus-Bellete/NLP_Project1/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
titles = ["Medicine", "Hospital", "Surgery", "Health","Heart","Vaccine",
    "Pharmacy", "Immunology", "Pathology","Treatment","Diabetes","Disease",
     "Therapy", "Dentistry","Kidney","Blood","Blood pressure","Virus",
    "Art", "Language", "Literature", "Philosophy",
    "Empire", "Space", "Environment","Color","Mountain",
    "Forest", "Cooking","Theology","Fashion",
    "History", "Geography", "Archaeology","government","Astronomy"]
    #create a data frame
df = pd.DataFrame(columns=["Text", "Label"])
dataArr = []
def fetc(title):
  url = 'https://en.wikipedia.org/w/api.php'
  params = {
        'action': 'query',
        'format': 'json',
        'titles': title,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
  }
  response = requests.get(url, params=params)
  data = response.json()
  return data
for topic in titles:
  data = fetc(topic)
  topic=next(iter(data['query']['pages'].values()))
  dataArr.append(topic['extract'][:500])

data = {"Text": dataArr, "Label": ["Medical"] * 18 + ["Non-Medical"] * 18}
try:
    df_added = pd.DataFrame(data)
    df = pd.concat([df, df_added], ignore_index=True)
   # print(df)
    df.to_csv('nlp_dataset.csv', index=False)
except ValueError as e:
    print(f"ValueError: {e}")


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install -y git
!git config --global user.email "dagnaw.kidus@gmail.com"
!git config --global user.name "Kidus Bellete"

In [ ]:
!mkdir /content/drive/MyDrive/colab_project
%cd /content/drive/MyDrive/colab_project
!git init

In [4]:
!cp /content/drive/MyDrive/Python/nlp_project.ipynb /content/drive/MyDrive/colab_project/

In [15]:
!git add .
!git commit -m "Initial commit"
!git remote add origin https://github.com/Kidus-Bellete/nlp-proj.git
!git branch -M main  # Replace "main" with "master" if using an older Git version
!git push -u origin main  # Replace "main" with your branch name


On branch main
nothing to commit, working tree clean
error: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address


# Python Natural Language Project Assignment I

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
#import pandas as pd

# read data
df = pd.read_csv('nlp_dataset.csv')
print("Original data distribution..")
print(df['Label'].value_counts())

# training and testing
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Label'],random_state=32)

# Fill missing values
X_train = X_train.fillna('')
X_test = X_test.fillna('')

# feature extraction
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Resampling using SMOTE
sampler = SMOTE(sampling_strategy='auto', k_neighbors=5)
X_train_resampled, y_train_resampled = sampler.fit_resample(X_train_tfidf, y_train)

# Model selection and training
model = MultinomialNB()
model.fit(X_train_resampled, y_train_resampled)

# model evaluation
predictions = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, predictions)
print(f"\nAccuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, predictions))

# making predictions
# data to predict
new_data = ["head ache is a type of disease ",
            "Ethiopia is the oldest city with many historical heritages.",
            "He has a chance to recover from his disease",
            "I love to see Italian historical places.",
            "This day reminds me so special memory of my graduation in the year of 2020. ",
            "How can I feel better and to improve my mood?",
            "does food changes my feeling?"
            #add any sentences to check its prediction value
            ]

new_data_tfidf = tfidf_vectorizer.transform(new_data)
new_predictions = model.predict(new_data_tfidf)

print("\nPredictions on new data:\n")

for text, prediction in zip(new_data, new_predictions):
    print(f"{text} - Predicted: {prediction}")


Original data distribution..
Medical        18
Non-Medical    18
Name: Label, dtype: int64

Accuracy: 88.88888888888889
Classification Report:
               precision    recall  f1-score   support

     Medical       1.00      0.83      0.91         6
 Non-Medical       0.75      1.00      0.86         3

    accuracy                           0.89         9
   macro avg       0.88      0.92      0.88         9
weighted avg       0.92      0.89      0.89         9


Predictions on new data:

head ache is a type of disease  - Predicted: Medical
Ethiopia is the oldest city with many historical heritages. - Predicted: Non-Medical
He has a chance to recover from his disease - Predicted: Medical
I love to see Italian historical places. - Predicted: Non-Medical
This day reminds me so special memory of my graduation in the year of 2020.  - Predicted: Non-Medical
How can I feel better and to improve my mood? - Predicted: Medical
does food changes my feeling? - Predicted: Non-Medical
